<a href="https://colab.research.google.com/github/Anannayjain/Recommendation-system/blob/main/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install LibRecommender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF  # pure data,
from libreco.evaluation import evaluate

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!wget -O dataset.zip "https://files.grouplens.org/datasets/movielens/ml-25m.zip"

--2024-05-11 04:35:51--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 249.84M  17.0MB/s    in 16s     

2024-05-11 04:36:09 (15.5 MB/s) - ‘dataset.zip’ saved [261978986/261978986]



In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
!cd ml-25m


In [ ]:
data = pd.read_csv('/content/ml-25m/ratings.csv')

In [ ]:
data.columns = ['user', 'item', 'label', 'time']

In [ ]:
data.head()

,user,item,label,time
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [ ]:
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1])

In [ ]:
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

In [ ]:
ncf = NCF(
    task="rating",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=16,
    n_epochs=5,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
)

In [ ]:
# monitor metrics on eval data during training
ncf.fit(
    train_data,
    neg_sampling=False, #for rating, this param is false else True
    verbose=2,
    eval_data=eval_data,
    metrics=["loss"],
)

# do final evaluation on test data
evaluate(
    model=ncf,
    data=test_data,
    neg_sampling=False,
    metrics=["loss"],
)
#for implicit feedback, metrics like precision@k, recall@k, ndcg can be used

/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
Instructions for updating:
Colocations handled automatically by placer.


Training start time: 2024-05-11 04:40:00


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
train: 100%|██████████| 19532/19532 [10:34<00:00, 30.81it/s]


Epoch 1 elapsed: 634.031s
	 train_loss: 0.7878


eval_pointwise: 100%|██████████| 306/306 [00:04<00:00, 74.57it/s]


...

Epoch 5 elapsed: 614.113s
	 train_loss: 0.5614


eval_pointwise: 100%|██████████| 306/306 [00:05<00:00, 61.20it/s]


	 eval rmse: 0.7857


eval_pointwise: 100%|██████████| 306/306 [00:07<00:00, 42.01it/s]


{'loss': 0.78440124}

In [ ]:
# predict preference of user 5755 to item 110
ncf.predict(user=5755, item=110)

# recommend 10items for user 5755
to_watch=ncf.recommend_user(user=5755, n_rec=10)

In [ ]:
movies=pd.read_csv('/content/ml-25m/movies.csv',index)

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies.set_index('movieId', inplace=True)


,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
209157,We (2018),Drama
209159,Window of the Soul (2001),Documentary
209163,Bad Poems (2018),Comedy|Drama


In [ ]:
print(movies.loc[2,'title'])

Jumanji (1995)


In [ ]:
to_watch[5755]

array([176601,   2691, 159817,  50005,  72360,  56141,  59387, 165641,
        63433, 171749])

In [ ]:
for id in to_watch[5755]:
  print(movies.loc[id,'title'])


Black Mirror
Legend of 1900, The (a.k.a. The Legend of the Pianist on the Ocean) (Leggenda del pianista sull'oceano) (1998)
Planet Earth (2006)
Curse of the Golden Flower (Man cheng jin dai huang jin jia) (2006)
Rocks (Das Rad) (2003)
Blame it on Fidel! (La faute à Fidel!) (2006)
Fall, The (2006)
Closet Monster (2016)
Farscape: The Peacekeeper Wars (2004)
Death Note: Desu nôto (2006–2007)
